In [ ]:
!uvicorn ChatTestAPI:app --host 127.0.0.1 --port 8000 --reload

In [ ]:
#First get a session
!curl -X POST "http://127.0.0.1:8000/start-session" \
     -H "Content-Type: application/json" \
     -d '{}'
# {
#   "session_id": "a2e8f3d1-5b6c-4d9e-8f1a-0b3c4d5e6f7a"
# }

#Then send a message
!curl -X POST http://127.0.0.1:8000/chat \
  -H "Content-Type: application/json" \
  -d '{"session_id": "YOUR_SESSION_ID", "message": "Hello!"}'
  
#With system prompt
!curl -X POST "http://127.0.0.1:8000/chat" \
     -H "Content-Type: application/json" \
     -d '{
           "session_id": "d290f1ee-6c54-4b01-90e6-d701748f0851",
           "message": "Tell me a joke.",
           "system_prompt": "You are a witty AI assistant specializing in jokes."
         }'
  
# First message Sample
!curl -X POST http://localhost:8000/chat \
  -H "Content-Type: application/json" \
  -d '{"session_id": "a2e8f3d1-5b6c-4d9e-8f1a-0b3c4d5e6f7a", "message": "Hello!"}'

# Subsequent messages Sample
!curl -X POST http://localhost:8000/chat \
  -H "Content-Type: application/json" \
  -d '{"session_id": "a2e8f3d1-5b6c-4d9e-8f1a-0b3c4d5e6f7a", "message": "What was my previous question?"}'
  
#Sample response
{
  "session_id": "d290f1ee-6c54-4b01-90e6-d701748f0851",
  "input": "Hello, how are you?",
  "response": "Hello! I'm doing well, thank you for asking. How can I assist you today?"
}

#RAG Chat
!curl -X POST "http://127.0.0.1:8000/chat-rag" \
     -H "Content-Type: application/json" \
     -d '{
           "session_id": "d290f1ee-6c54-4b01-90e6-d701748f0851",
           "vectorstore_id": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a",
           "message": "Tell me about the latest advancements in AI."
         }'
         
#With system prompt
!curl -X POST "http://127.0.0.1:8000/chat-rag" \
     -H "Content-Type: application/json" \
     -d '{
           "session_id": "d290f1ee-6c54-4b01-90e6-d701748f0851",
           "vectorstore_id": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a",
           "message": "Tell me about the latest advancements in AI.",
           "system_prompt": "You are a knowledgeable AI researcher."
         }'
         
#Sample response
{
  "session_id": "d290f1ee-6c54-4b01-90e6-d701748f0851",
  "vectorstore_id": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a",
  "input": "Tell me about the latest advancements in AI.",
  "response": "Recent advancements in AI include significant improvements in natural language processing, computer vision, and reinforcement learning. Models like GPT-4 have demonstrated remarkable capabilities in understanding and generating human-like text, while advancements in computer vision have led to more accurate image and video recognition systems. Additionally, reinforcement learning has seen breakthroughs in areas such as game playing and autonomous systems."
}
  
#Delete a session
!curl -X DELETE "http://localhost:8000/delete-session" \
  -H "Content-Type: application/json" \
  -d '{"session_id": "abc123"}'
  
#Sample response
{
  "detail": "Session deleted successfully",
  "session_id": "abc123",
  "remaining_sessions": 2
}

In [ ]:
#Attempting to chat with a non-existent session:
!curl -X POST "http://127.0.0.1:8000/chat-rag" \
     -H "Content-Type: application/json" \
     -d '{
           "session_id": "non-existent-session-id",
           "vectorstore_id": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a",
           "message": "Test message"
         }'

#Sample response
{
  "detail": "Session not found"
}

#Attempting to use a non-existent vector store:
!curl -X POST "http://127.0.0.1:8000/chat-rag" \
     -H "Content-Type: application/json" \
     -d '{
           "session_id": "d290f1ee-6c54-4b01-90e6-d701748f0851",
           "vectorstore_id": "non-existent-vectorstore-id",
           "message": "Test message"
         }'

#Sample response
{
  "detail": "Pinecone index 'non-existent-vectorstore-id' not found."
}

In [ ]:
#Sample Client
import requests
import uuid

class ChatClient:
    def __init__(self, base_url="http://localhost:8000"):
        self.base_url = base_url
        self.session_id = None
    
    def start_session(self):
        response = requests.post(f"{self.base_url}/start-session")
        response.raise_for_status()
        self.session_id = response.json()["session_id"]
        return self.session_id
    
    def chat(self, message):
        if not self.session_id:
            raise ValueError("No active session. Call start_session() first.")
            
        payload = {
            "session_id": self.session_id,
            "message": message
        }
        response = requests.post(f"{self.base_url}/chat", json=payload)
        response.raise_for_status()
        return response.json()["response"]

# Usage example
if __name__ == "__main__":
    client = ChatClient()
    client.start_session()
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == "/exit":
            break
            
        response = client.chat(user_input)
        print(f"AI: {response}")

In [1]:
#Create a new vector store
!curl -X POST "http://127.0.0.1:8000/create-vector-store" \
     -H "Content-Type: application/json" \
     -d '{
           "text": "Your text goes here. This could be any text you want to convert into a vector store.",
           "chunk_size": 500,
           "overlap": 50
         }'

#Sample Response
{
  "index_name": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a",
  "dimension": 1536,
  "metadata": {
    "index_name": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a",
    "environment": "us-west1-gcp",
    "dimension": 1536,
    "vector_count": 1 
  }
}

#Delete a vector store
!curl -X POST "http://127.0.0.1:8000/delete-vector-store" \
     -H "Content-Type: application/json" \
     -d '{
           "index_name": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a"
         }'


#Sample Response
{
  "message": "Pinecone index 'text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a' has been deleted successfully.",
  "index_name": "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a"
}